In [11]:
from elasticsearch import Elasticsearch

In [12]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [13]:
if es.indices.exists('es0'):
    es.indices.delete('es0')
es.indices.create('es0')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [14]:
'''
Analyzed text fields take term positions into account, in order to be able to support proximity or phrase queries.
When indexing text fields with multiple values a "fake" gap is added between the values to prevent most phrase queries from matching across the values.
The size of this gap is configured using position_increment_gap and defaults to 100.
'''
es.index(index='es0', id=0, body={
    "names": ["John Abraham", "Lincoln Smith"]
})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '0',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [15]:
# Abraham Lincoln不在查询中
es.search(body={
    "query": {
        "match_phrase": {
            "names": {
                "query": "Abraham Lincoln"
            }
        }
    }
}, index='es0')

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [16]:
# This phrase query matches our document, even though Abraham and Lincoln are in separate strings, because slop > position_increment_gap.
es.search(body={
    "query": {
        "match_phrase": {
            "names": {
                "query": "Abraham Lincoln",
                "slop": 101
            }
        }
    }
}, index='es0')

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.010358453,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.010358453,
    '_source': {'names': ['John Abraham', 'Lincoln Smith']}}]}}

In [17]:
if es.indices.exists('es1'):
    es.indices.delete('es1')
es.indices.create('es1', body={
    "mappings": {
        "properties": {
            "names": {
                "type": "text",
                "position_increment_gap": 0
            }
        }
    }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es1'}

In [19]:
# The first term in the next array element will be 0 terms apart from the last term in the previous array element.
es.index(index='es1', id=0, body={
    "names": ["John Abraham", "Lincoln Smith"]
})

# The phrase query matches our document which is weird, but its what we asked for in the mapping.
es.search(body={
    "query": {
        "match_phrase": {
            "names": {
                "query": "Abraham Lincoln"
            }
        }
    }
}, index='es1')

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.5753642,
  'hits': [{'_index': 'es1',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.5753642,
    '_source': {'names': ['John Abraham', 'Lincoln Smith']}}]}}